In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
dataset_dict = pd.read_csv('dict_csv.csv')

In [3]:
dataset_dict.drop(columns=[
    'vorder',
    'var_code',
    'Area',
    'Collection \nInstrument',
    'Questionnaire \n- general section',
    'Questionnaire\n- subsection/ scale',
    'CAPI code',
    'Derived variable',
    'Requested\n(y = yes)',
    'Research Purpose\n(e = exposure,\nc = confounder,\no = outcome)',
    'Newly Requested (Amendment)\n(y = yes)'
], inplace=True)

dataset_dict.rename(columns={'Order_ID\n(internal use)': 'Order_ID'}, inplace=True)

In [4]:
dataset_dict.head()

,Order_ID,Dataset name,Sweep / Year / Age,Topic,Respondent,About whom,Variable label,Value labels,Variable name
0,14755,mcs6_cm_interview.sav,Sweep 6 - 2015/6 - 14 years old,Administration,NaN,Cohort member,Cohort Member number within an MCS family,(1.0) 1st Cohort Member of the family (2.0) 2n...,FCNUM00
1,14756,mcs6_cm_interview.sav,Sweep 6 - 2015/6 - 14 years old,Gender,Cohort Member,Cohort Member,CM Sex,(-1.0) Not known/Not applicable (1.0) M...,FCCSEX00
2,14757,mcs6_cm_interview.sav,Sweep 6 - 2015/6 - 14 years old,Age,Cohort Member,Cohort Member,CM DOB (month),(-1.0) Not applicable (1.0) January (2.0) Feb...,FCCDBM00
3,14758,mcs6_cm_interview.sav,Sweep 6 - 2015/6 - 14 years old,Age,Cohort Member,Cohort Member,CM DOB (year),(-1.0) Not applicable,FCCDBY00
4,14759,mcs6_cm_interview.sav,Sweep 6 - 2015/6 - 14 years old,Age,Cohort Member,Cohort Member,CM Age last birthday,(-1.0) Not applicable,FCCAGE00


In [5]:
raw_data = pd.read_csv('../../raw_data/sweep_6/UKDA-8156-tab/tab/mcs6_cm_interview.tab', sep='\t')

In [6]:
raw_data.head()

,MCSID,FCNUM00,FCCSEX00,FCCDBM00,FCCDBY00,FCCAGE00,FCVERSF0,FCINTROA,FCCHIC0A,FCCHIC0B,...,FCCARR02_TR3,FCCARR03_TR3,FCCARR04_TR3,FCCARR05_TR3,FCCARR06_TR3,FCCARR07_TR3,FCCARR08_TR3,FCCARR09_TR3,FCCARR10_TR3,FCCARR11_TR3
0,M10002P,1,1,3,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,M10007U,1,1,10,2000,14,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,M10015U,1,2,5,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,M10016V,1,1,11,2000,14,102,1,1,0,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,M10018X,1,2,7,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [7]:
raw_cols = raw_data.columns.values[1:]
raw_cols

array(['FCNUM00', 'FCCSEX00', 'FCCDBM00', 'FCCDBY00', 'FCCAGE00',
       'FCVERSF0', 'FCINTROA', 'FCCHIC0A', 'FCCHIC0B', 'FCCHIC0C',
       'FCCHIC0D', 'FCCHIC0E', 'FCCHAC0A', 'FCCHAC0B', 'FCCHAC0C',
       'FCCHAC0D', 'FCCHAC0E', 'FCHTCM00', 'FCHTCM1D', 'FCHTCMOR',
       'FCHTCMPE', 'FCHTFT00', 'FCHTIN00', 'FCHTCCKA', 'FCHTCCKB',
       'FCHTCCKC', 'FCHTCCKD', 'FCHKHTCK', 'FCUNHT00', 'FCHTAT00',
       'FCHTATCK', 'FCHTM100', 'FCHTM200', 'FCHTTMCK', 'FCHTRL0A',
       'FCHTRL0B', 'FCHTRL0C', 'FCHTRL0D', 'FCHTRL0E', 'FCHTRL0F',
       'FCHTRL0G', 'FCHTRL0H', 'FCHTRL0I', 'FCHTRL0J', 'FCHTRL0K',
       'FCHTRL0L', 'FCHTRL0M', 'FCHTRL0N', 'FCHTEL00', 'FCBFCK00',
       'FCBFCKCK', 'FCWTCM00', 'FCWTCM1D', 'FCWTCMOR', 'FCWTCMPE',
       'FCWTST00', 'FCWTPO00', 'FCWTC1CK', 'FCWTC2CK', 'FCWTC3CK',
       'FCWTC4CK', 'FCWTCMCK', 'FCWTUN00', 'FCBFPC00', 'FCBFPCD0',
       'FCBFPCCK', 'FCBDFMCK', 'FCFEET00', 'FCNOBF00', 'FCWTAT00',
       'FCWTATCK', 'FCWTSC00', 'FCWTRL0A', 'FCWTRL0B', 'FCWTRL0

In [48]:
processed_cols = [dataset_dict[dataset_dict['Variable name'] == col].iloc[0]['Variable label'].strip().replace(' ', '_') for col in raw_cols]
processed_cols

processed_data = raw_data.copy()
for old, new in zip(raw_cols, processed_cols):
    processed_data.rename(columns={old: new}, inplace=True)


processed_data

,MCSID,Cohort_Member_number_within_an_MCS_family,CM_Sex,CM_DOB_(month),CM_DOB_(year),CM_Age_last_birthday,PM_IWinf_-_Script_version,PhysMeasIWinfo_entry_flag,"IWR:_Guardian_consent_for_physical_measurements_[Yes,_consent_for_all]","IWR:_Guardian_consent_for_physical_measurements_MC_[Yes,_consent_for_height]",...,Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R2_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R3_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R4_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R5_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R6_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R7_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R8_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R9_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R10____[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R11____[truncated_at_3_chars]
0,M10002P,1,1,3,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,M10007U,1,1,10,2000,14,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,M10015U,1,2,5,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,M10016V,1,1,11,2000,14,102,1,1,0,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,M10018X,1,2,7,2001,13,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,M35868P,1,1,11,2000,14,103,1,1,0,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
11855,M35869Q,1,2,10,2001,14,103,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11856,M35872K,1,2,12,2000,14,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11857,M35873L,1,2,10,2000,14,102,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [9]:
mcs6_option_map = {}

for i, feature in dataset_dict.iterrows():
    options = feature['Value labels']
    option_values = {}
    pattern = r'\((-?\d+\.\d+)\)\s*([^()]+)'
    try:
        matches = re.findall(pattern, options)
    except:
        continue

    for match in matches:
        key = match[0]
        value = str(match[1]).strip()
        option_values[key] = value

    mcs6_option_map[feature['Variable name']] = option_values

with open('mcs6_option_map', 'wb') as file:
    pickle.dump(mcs6_option_map, file)


In [10]:
mcs6_option_map['FCCDBY00']
raw_data['FCCDBY00']

0        2001
1        2000
2        2001
3        2000
4        2001
         ... 
11854    2000
11855    2001
11856    2000
11857    2000
11858    2001
Name: FCCDBY00, Length: 11859, dtype: int64

In [11]:
with open('mcs6_option_map', 'rb') as file:
    mcs6_option_map = pickle.load(file)

In [12]:
final_df = raw_data.copy()

for i, row in final_df.iterrows():
    for variable, options in mcs6_option_map.items():
        try:
            final_df.loc[i, variable] = options[str(float(row[variable]))]
        except:
            pass
    print(f"{i} / {len(final_df)}")

0 / 11859
1 / 11859
2 / 11859
3 / 11859
4 / 11859
5 / 11859
6 / 11859
7 / 11859
8 / 11859
9 / 11859
10 / 11859
11 / 11859
12 / 11859
13 / 11859
14 / 11859
15 / 11859
16 / 11859
17 / 11859
18 / 11859
19 / 11859
20 / 11859
21 / 11859
22 / 11859
23 / 11859
24 / 11859
25 / 11859
26 / 11859
27 / 11859
28 / 11859
29 / 11859
30 / 11859
31 / 11859
32 / 11859
33 / 11859
34 / 11859
35 / 11859
36 / 11859
37 / 11859
38 / 11859
39 / 11859
40 / 11859
41 / 11859
42 / 11859
43 / 11859
44 / 11859
45 / 11859
46 / 11859
47 / 11859
48 / 11859
49 / 11859
50 / 11859
51 / 11859
52 / 11859
53 / 11859
54 / 11859
55 / 11859
56 / 11859
57 / 11859
58 / 11859
59 / 11859
60 / 11859
61 / 11859
62 / 11859
63 / 11859
64 / 11859
65 / 11859
66 / 11859
67 / 11859
68 / 11859
69 / 11859
70 / 11859
71 / 11859
72 / 11859
73 / 11859
74 / 11859
75 / 11859
76 / 11859
77 / 11859
78 / 11859
79 / 11859
80 / 11859
81 / 11859
82 / 11859
83 / 11859
84 / 11859
85 / 11859
86 / 11859
87 / 11859
88 / 11859
89 / 11859
90 / 11859
91 / 1185

In [13]:
final_df

,MCSID,FCNUM00,FCCSEX00,FCCDBM00,FCCDBY00,FCCAGE00,FCVERSF0,FCINTROA,FCCHIC0A,FCCHIC0B,...,FCCARR02_TR3,FCCARR03_TR3,FCCARR04_TR3,FCCARR05_TR3,FCCARR06_TR3,FCCARR07_TR3,FCCARR08_TR3,FCCARR09_TR3,FCCARR10_TR3,FCCARR11_TR3
0,M10002P,1st Cohort Member of the family,Male,March,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,M10007U,1st Cohort Member of the family,Male,October,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,M10015U,1st Cohort Member of the family,Female,May,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,M10016V,1st Cohort Member of the family,Male,November,2000,14,102,Answered,Yes,No,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,M10018X,1st Cohort Member of the family,Female,July,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,M35868P,1st Cohort Member of the family,Male,November,2000,14,103,Answered,Yes,No,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
11855,M35869Q,1st Cohort Member of the family,Female,October,2001,14,103,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11856,M35872K,1st Cohort Member of the family,Female,December,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11857,M35873L,1st Cohort Member of the family,Female,October,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [112]:
processed_cols = [dataset_dict[dataset_dict['Variable name'] == col].iloc[0]['Variable label'].strip().replace(' ', '_') for col in raw_cols]
processed_cols

processed_data = final_df.copy()
for old, new in zip(raw_cols, processed_cols):
    processed_data.rename(columns={old: new}, inplace=True)


processed_data

,MCSID,Cohort_Member_number_within_an_MCS_family,CM_Sex,CM_DOB_(month),CM_DOB_(year),CM_Age_last_birthday,PM_IWinf_-_Script_version,PhysMeasIWinfo_entry_flag,"IWR:_Guardian_consent_for_physical_measurements_[Yes,_consent_for_all]","IWR:_Guardian_consent_for_physical_measurements_MC_[Yes,_consent_for_height]",...,Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R2_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R3_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R4_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R5_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R6_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R7_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R8_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R9_[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R10____[truncated_at_3_chars],Aspirations:_what_CM_would_like_to_be_when_grow_up_[SOC_code]_R11____[truncated_at_3_chars]
0,M10002P,1st Cohort Member of the family,Male,March,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,M10007U,1st Cohort Member of the family,Male,October,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,M10015U,1st Cohort Member of the family,Female,May,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,M10016V,1st Cohort Member of the family,Male,November,2000,14,102,Answered,Yes,No,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,M10018X,1st Cohort Member of the family,Female,July,2001,13,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,M35868P,1st Cohort Member of the family,Male,November,2000,14,103,Answered,Yes,No,...,344,-1,-1,-1,-1,-1,-1,-1,-1,-1
11855,M35869Q,1st Cohort Member of the family,Female,October,2001,14,103,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11856,M35872K,1st Cohort Member of the family,Female,December,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11857,M35873L,1st Cohort Member of the family,Female,October,2000,14,102,Answered,Yes,No,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [113]:
with open('processed_data', 'wb') as file:
    pickle.dump(processed_data, file)

In [14]:
# save df with original variable names
with open('../../raw_data/processed_data_original_names', 'wb') as file:
    pickle.dump(final_df, file)

In [4]:
with open('processed_data', 'rb') as file:
    processed_data = pickle.load(file)

In [18]:
processed_data['Original WTCM value pre Edit'.replace('_', ' ')].value_counts()

KeyError: 'Original WTCM value pre Edit'

In [33]:
var = 'FCOPWE0C'

col = final_df[var]

col.value_counts()

Yes                     9137
Not applicable          2435
No                       277
Don't know                 9
Don't want to answer       1
Name: FCOPWE0C, dtype: int64